<a href="https://colab.research.google.com/github/magomeda228/Colab_Google_ML_H-W/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22homework_4_week%5Breu_ml_school%5D_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <center>**ML School by REU Data Scinece Club**</center>

---
## <center> Неделя №4 </center>


На этой неделе вам предстоит написать свою логистическую регрессию на чистом numpy и оптимизировать ее спомощью градиентного спуска. Сторонние биболетки (например, если знакомы с Torch) для вычисления использовать нельзя

Импортируем библиотеки, которые могут вам понадобиться, а затем загрузим данные

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

Мы возьмем синтетические данные (созданные нами искутсвенно). Эти данные не нуждаются в обратботке

In [2]:
!git clone https://github.com/MeinRespect/REU-Ml-SCHOOL.git

Cloning into 'REU-Ml-SCHOOL'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 20 (delta 4), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (20/20), 14.94 MiB | 5.11 MiB/s, done.
Resolving deltas: 100% (4/4), done.
Updating files: 100% (7/7), done.


In [3]:
df = pd.read_csv('/content/REU-Ml-SCHOOL/simple_data_for_linear_regression.csv',
                 index_col='Unnamed: 0')

df.head(10)

,1,2,3,4,5,6,7,8,9,Target
0,-0.825654,0.996750,2.172271,0.296957,-0.463338,-0.349884,2.511452,-0.928644,-2.778066,1
1,-0.557834,0.160590,0.286331,-0.041377,-1.448752,0.027275,0.048385,-0.764340,-0.049713,1
2,0.309318,-0.638823,0.937555,1.098689,2.917309,-0.305654,0.647202,-0.108898,-0.740039,0
3,0.814625,-1.468853,-2.514925,-0.247016,0.276103,-0.445736,0.393316,1.142564,-0.478847,0
4,0.529211,1.362208,0.026132,1.244459,0.165382,0.625437,-1.267929,0.604447,1.452787,1
5,-0.140911,-1.729456,-0.254853,0.259032,-0.661892,-0.691096,1.171270,0.186937,-1.354727,0
6,-0.962663,-0.990447,-1.106779,1.147607,-1.251957,0.127369,-1.557314,0.895921,1.712605,0
7,2.018122,0.725902,0.144987,-0.316239,-0.621207,0.092930,0.348007,-2.453892,-0.369115,1
8,-0.017818,0.785941,0.457947,0.146765,-0.600139,0.095527,0.398463,0.572963,-0.423850,0
9,-0.614641,1.036744,0.581185,-1.217846,0.120342,0.505202,-1.089338,-0.163125,1.244560,1


Теперь нужно разделить на train и test c помощью train_test_split из sklearn, мы его уже импортировали.
В качестве test_size выберите 0.25, а random_state = 1. Random state указывает, как должны генериться случаные числа. Указав его, мы будем всегда получать одни и те же значения, это поможет нам воспроизводить результат заново

In [4]:
x=df.drop(columns='Target')
y=df['Target']
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25,random_state=1)

Теперь все готово к написанию самой модели

# **Задание 1. Градиентный спуск (9 баллов)**

Наша модель должна быть в матричном виде. Bias будет включен в матрицу весов. Реализуем градиентный спуск.

Вот несколько нужных формул:

p - наши предсказанные значения

1. Градиентный спуск:   $$\nabla L(w) = - lr \frac{\partial}{\partial w} L(w)$$

2. Бинарная кросс энтропия:   $$ L(y, p) = -[y\log(p) + (1-y)\log(1-p)] $$

3. Сигмоидное преобразование:  $$ p = \sigma(z) = \frac{1}{1 + e^{-z}} $$

4. Регрессия $$ {z} = X w $$

В нашем случае нам нужно получить производную функции потерь (Бинарная кросс энтропия) по W. Это и есть наши веса или коэффицниенты. Производная в данном случае будет сложная, и именно ее значения мы будем домножать на learning rate. Вы можете вычислить на прямую: $$ grad = \frac{\partial L(w)}{\partial w} $$
Но это тяжело, поэтому можно также сделать по-другому, воспользовавшись цепным правилом(Chain rule): $$ grad = \frac{\partial L( p )}{\partial p} \frac{\partial \sigma(z) }{\partial z} \frac{\partial z }{\partial w}$$

Получится то же самое, но посчитать будет легче. Просто так получить градиент не выйдет, поле нахождения производных нужно сократить некоторые части (кстати, максимально сократить громоздкую формулу можно к одному перемножению и одному вычитанию).

In [11]:
def gradient_descent(X, weights, y_pred, y_real, learning_rate):

    m, n = X.shape

    grad=(1/m)*X.T @(y_pred-y_real)
    weights-=learning_rate*grad

    return weights

def sigmoid(z):
    return 1/(1+np.exp(-z))

In [ ]:
# Ваш код тут

Эта часть кода покажет правильно ли вы реализовали градиентный спуск. Нужно просто запустить её, и если все было сделано правильно, то ошибок не произойдет. Возможен и вариант, что ваш градиентный спуск не прошел тест, но коэффициенты получились практически такими же. В этом случае попробуйте пойти дальше: обучить модель и посмотреть на результат обучения.

In [12]:
X = np.array([[1, 2], [3, 4], [5, 6]])
weights = np.array([0.1, 0.2])
y_real = np.array([3, 7, 11])

y_pred = X @ weights

learning_rate = 0.01
weights_updated = gradient_descent(X, weights, y_pred, y_real, learning_rate)
expected_weights_updated = np.array([0.32233333, 0.48133333])
np.testing.assert_allclose(weights_updated, expected_weights_updated, rtol=1e-5)

# **Задание 2. Логистическая регрессия (5 баллов)**

Не забудьте, что у нас все в матричном виде, поэтому X и матрицу весов нужно немного изменить. В качестве метрики используйте f1_score. Мы пройдем ее на следующей неделе, пока нужно знать только, что чем ближе она к 1, тем лучше.
Применяется она так f1_score(y_true, y_pred), где y_true - правильные значение, а y_pred - значения, предсказанные моделью.

Если вам нужно, то краткий экскурс в ООП вы найдете тут: https://pythonchik.ru/osnovy/osnovy-oop-v-python-klassy-obekty-metody

In [51]:
class Logistic_Regression():
    def __init__(self, num_of_columns):
        # Инициализируем веса с учетом bias
        self.weights = np.random.randn(num_of_columns + 1, 1)

    def train(self, X, y, learning_rate, iterations_num):
        # Добавляем bias в X
        X_with_bias = np.hstack([np.ones((X.shape[0], 1)), X])
        y = y.reshape(-1, 1)  # Изменяем размерность y на (n_samples, 1)

        for step in range(iterations_num):
            z = X_with_bias @ self.weights  # Вычисляем z
            y_pred = self.sigmoid(z)  # Получаем предсказания

            # Вычисляем градиент
            error = y_pred - y  # y теперь имеет правильную размерность
            grad = (1 / X.shape[0]) * (X_with_bias.T @ error)
            self.weights -= learning_rate * grad  # Обновляем веса

            # Выводим ошибку на первой и последней итерации
            if step == 0 or step == iterations_num - 1:
                loss = self.binary_cross_entropy(y, y_pred)
                print(f"Итерация {step}, ошибка: {loss}")

        # Округляем предсказания для вычисления f1
        y_pred_rounded = np.where(y_pred >= 0.5, 1, 0)
        return y_pred_rounded.flatten()  # Возвращаем одномерный массив

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def binary_cross_entropy(self, y, y_pred):
        # Функция потерь
        return -np.mean(y * np.log(y_pred + 1e-15) + (1 - y) * np.log(1 - y_pred + 1e-15))

    def predict(self, X):
        X_with_bias = np.hstack([np.ones((X.shape[0], 1)), X])  # Добавляем bias
        z = X_with_bias @ self.weights
        preds = self.sigmoid(z)
        return np.where(preds > 0.5, 1, 0)

Теперь обучим модель и проверим ее на тестовой выборке с помощью f1_score. Она должна быть больше 0.85 на тестовой выборке. Если у вас меньше, то вам нужно поменять параметры, например, увеличить learning rate или количество итераций

In [58]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
if y_train.ndim == 1:
    y_train = y_train.reshape(-1, 1)
# Масштабируем обучающие данные
x_train_scaled = scaler.fit_transform(x_train)

# Масштабируем тестовые данные
x_test_scaled = scaler.transform(x_test)
model = Logistic_Regression(num_of_columns=x_train.shape[1])
y_pred = model.train(x_train_scaled, y_train, learning_rate=0.001, iterations_num=1000)

# Оценка вашей модели
f1 = f1_score(y_test, y_pred)
print(f"F1 Score на тестовой выборке (ваша модель): {f1}")

ValueError: operands could not be broadcast together with shapes (300,1) (800,1) 

# **Задание 3. Логистическая регрессия в Sklearn (6 баллов)**

In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

Теперь, написав свою модель, ее можно сравнить с моделью из Sklearn, обучите LogisticRegression и также проверьте на тестовой выборке по f1_score


In [42]:
X = np.random.rand(1000, 10)
y = np.random.randint(0, 2, 1000)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LogisticRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
f1=f1_score(y_test,y_pred)
print(f"F1 Score (Logistic Regression): {f1:.4f}")

F1 Score (Logistic Regression): 0.4520


Логичесческая регрессия модель хорошая, но не лучшая. Давайте теперь попробуем построить SVM из Sklearn. Все также: обучите ее и проверьте на тестовой выборке.

In [28]:
X = np.random.rand(1000, 10)
y = np.random.randint(0, 2, 1000)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model=SVC(kernel='linear')
model.fit(X_train,y_train)
y_pred=model.predict(X_test)
f1=f1_score(y_test,y_pred)
print(f"F1 Score (SVM): {f1:.4f}")

F1 Score (SVM): 0.5249
